<h1>Crypto Market Simulator</h1>

In [1]:
import asyncio
import ujson
import time
import sys 
import pandas as pd

from gate_ws import Configuration, Connection, WebSocketResponse
from gate_ws.spot import SpotPublicTradeChannel, SpotOrderBookChannel, SpotOrderBookUpdateChannel, SpotBookTickerChannel

In [2]:
NUM_OF_SNAPSHOTS = 5
curr_snapshot = 0
orders = []

<h2>Function to get orderbook data in json format</h2>

In [37]:
def print_message(conn: Connection, response: WebSocketResponse):
    global curr_snapshot
    global NUM_OF_SNAPSHOTS
    global orders
    if response.error:
        print('error returned: ', response.error)
        conn.close()
        return
    
    data = response.result
    if list(data.keys())[0] != "status":
        orders.append(data)
        curr_snapshot += 1
    
    if curr_snapshot >= NUM_OF_SNAPSHOTS:
        conn.close()
        return
        
async def main():
    channel_name = "BTC_USD"
    depth = "20"
    update_rate = "1000ms"
    
    filename = "orderbook.json"
    
    conn = Connection(Configuration())
    
    channel = SpotOrderBookChannel(conn, print_message)
    
    channel.subscribe([channel_name, depth, update_rate])

    await conn.run()
    
    with open(filename, "w") as file:
        data = ujson.dumps(orders, indent=4)
        print(data)
        file.write(data)

if __name__ == '__main__':
    start_time = time.perf_counter()
    await main()
    end_time = time.perf_counter()
    print(f"Time: {end_time - start_time}")

[
    {
        "t": 1673293859193,
        "lastUpdateId": 913788000,
        "s": "BTC_USD",
        "bids": [
            [
                "17319.27",
                "0.003"
            ],
            [
                "17319.25",
                "0.003"
            ],
            [
                "17319.23",
                "0.003"
            ],
            [
                "17315",
                "0.2959"
            ],
            [
                "17310.73",
                "0.0016"
            ],
            [
                "17305.81",
                "0.0004"
            ],
            [
                "17305.8",
                "0.5567"
            ],
            [
                "17291.51",
                "0.5454"
            ],
            [
                "17280.16",
                "0.0014"
            ],
            [
                "17280.15",
                "0.0024"
            ],
            [
                "17258.08",
                "1.8791"
       

In [45]:
df = pd.read_json("orderbook.json")

df

,t,lastUpdateId,s,bids,asks
0,1673293859193,913788000,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328...."
1,1673293860771,913788035,BTC_USD,"[[17319.28, 0.0008], [17319.27, 0.003], [17319...","[[17328.14, 0.003], [17328.16, 0.003], [17328...."
2,1673293861593,913788057,BTC_USD,"[[17319.28, 0.003], [17319.27, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328...."
3,1673293862283,913788058,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328...."
4,1673293863290,913788067,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328...."


In [46]:
df2 = pd.DataFrame(df)

df2[[f'bid_{i}' for i in range(20)]] = pd.DataFrame(df2.bids.tolist(), index= df2.index)
df2[[f'ask_{i}' for i in range(20)]] = pd.DataFrame(df2.asks.tolist(), index= df2.index)

df2

,t,lastUpdateId,s,bids,asks,bid_0,bid_1,bid_2,bid_3,bid_4,...,ask_10,ask_11,ask_12,ask_13,ask_14,ask_15,ask_16,ask_17,ask_18,ask_19
0,1673293859193,913788000,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]","[17310.73, 0.0016]",...,"[17331.28, 0.003]","[17331.3, 0.003]","[17331.32, 0.003]","[17331.34, 0.003]","[17331.36, 0.003]","[17331.37, 0.003]","[17332.29, 0.003]","[17332.3, 0.2959]","[17333.99, 0.003]","[17334.01, 0.003]"
1,1673293860771,913788035,BTC_USD,"[[17319.28, 0.0008], [17319.27, 0.003], [17319...","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.28, 0.0008]","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]",...,"[17331.28, 0.003]","[17331.3, 0.003]","[17331.32, 0.003]","[17331.34, 0.003]","[17331.36, 0.003]","[17331.37, 0.003]","[17332.29, 0.003]","[17332.3, 0.2959]","[17333.99, 0.003]","[17334.01, 0.003]"
2,1673293861593,913788057,BTC_USD,"[[17319.28, 0.003], [17319.27, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.28, 0.003]","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]",...,"[17331.28, 0.003]","[17331.3, 0.003]","[17331.32, 0.003]","[17331.34, 0.003]","[17331.36, 0.003]","[17331.37, 0.003]","[17332.29, 0.003]","[17332.3, 0.2959]","[17333.99, 0.003]","[17334.01, 0.003]"
3,1673293862283,913788058,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]","[17310.73, 0.0016]",...,"[17331.28, 0.003]","[17331.3, 0.003]","[17331.32, 0.003]","[17331.34, 0.003]","[17331.36, 0.003]","[17331.37, 0.003]","[17332.29, 0.003]","[17332.3, 0.2959]","[17333.99, 0.003]","[17334.01, 0.003]"
4,1673293863290,913788067,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]","[17310.73, 0.0016]",...,"[17331.28, 0.003]","[17331.3, 0.003]","[17331.32, 0.003]","[17331.34, 0.003]","[17331.36, 0.003]","[17331.37, 0.003]","[17332.29, 0.003]","[17332.3, 0.2959]","[17333.99, 0.003]","[17334.01, 0.003]"


In [50]:
for i in range(20):
    df2[[f'bid_{i}_px', f"bid_{i}_qty"]] = pd.DataFrame(df2[f"bid_{i}"].tolist(), index= df2.index).apply(pd.to_numeric, errors = 'coerce')
    df2[[f'ask_{i}_px', f"ask_{i}_qty"]] = pd.DataFrame(df2[f"ask_{i}"].tolist(), index= df2.index).apply(pd.to_numeric, errors = 'coerce')

# for i in range(20):
#     df2[f"bid_{i}"].apply(lambda x: pd.Series(str(x).split(",").to_numeric))
#     df2[f"ask_{i}"].apply(lambda x: pd.Series(str(x).split(",").to_numeric))


#     df2[[f'bid_{i}_px', f"bid_{i}_qty"]] = df[[f'bid_{i}_px', f"bid_{i}_qty"]].apply(pd.to_numeric, errors = 'coerce')
#     df2[[f'ask_{i}_px', f"ask_{i}_qty"]] = df[[f'ask_{i}_px', f"ask_{i}_qty"]].apply(pd.to_numeric, errors = 'coerce')
# print(df2["bids"])
# print(df2["asks"])
df2

/Users/kritiksatija/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,t,lastUpdateId,s,bids,asks,bid_0,bid_1,bid_2,bid_3,bid_4,...,ask_17_px,ask_17_qty,bid_18_px,bid_18_qty,ask_18_px,ask_18_qty,bid_19_px,bid_19_qty,ask_19_px,ask_19_qty
0,1673293859193,913788000,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]","[17310.73, 0.0016]",...,17332.3,0.2959,17100.00,0.0016,17333.99,0.003,17088.98,0.0146,17334.01,0.003
1,1673293860771,913788035,BTC_USD,"[[17319.28, 0.0008], [17319.27, 0.003], [17319...","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.28, 0.0008]","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]",...,17332.3,0.2959,17139.49,0.0024,17333.99,0.003,17100.00,0.0016,17334.01,0.003
2,1673293861593,913788057,BTC_USD,"[[17319.28, 0.003], [17319.27, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.28, 0.003]","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]",...,17332.3,0.2959,17139.49,0.0024,17333.99,0.003,17100.00,0.0016,17334.01,0.003
3,1673293862283,913788058,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]","[17310.73, 0.0016]",...,17332.3,0.2959,17100.00,0.0016,17333.99,0.003,17088.98,0.0146,17334.01,0.003
4,1673293863290,913788067,BTC_USD,"[[17319.27, 0.003], [17319.25, 0.003], [17319....","[[17328.14, 0.003], [17328.16, 0.003], [17328....","[17319.27, 0.003]","[17319.25, 0.003]","[17319.23, 0.003]","[17315, 0.2959]","[17310.73, 0.0016]",...,17332.3,0.2959,17100.00,0.0016,17333.99,0.003,17088.98,0.0146,17334.01,0.003


<h2>Final dataframe format</h2>

In [21]:
#Remove unnecessary columns
df2.drop(columns = [f'bid_{i}' for i in range(20)]+[f'ask_{i}' for i in range(20)]+["bids","asks", "lastUpdateId"], axis=1, inplace = True)

df2

,t,s,bid_0_px,bid_0_qty,ask_0_px,ask_0_qty,bid_1_px,bid_1_qty,ask_1_px,ask_1_qty,...,ask_17_px,ask_17_qty,bid_18_px,bid_18_qty,ask_18_px,ask_18_qty,bid_19_px,bid_19_qty,ask_19_px,ask_19_qty
0,1671462567378,BTC_USD,16695.22,0.0063,16712.48,0.0150,16695.21,0.0250,16712.49,0.0250,...,16733.07,0.005,16588.19,0.0024,16733.10,0.005,16582.92,0.0326,16733.12,0.005
1,1671462568581,BTC_USD,16695.21,0.0250,16712.49,0.0250,16695.20,0.2883,16712.50,0.2883,...,16733.10,0.005,16588.19,0.0024,16733.12,0.005,16582.92,0.0326,16733.19,0.005
2,1671462569839,BTC_USD,16701.09,0.0002,16712.48,0.0035,16695.22,0.0080,16712.49,0.0250,...,16733.07,0.005,16596.20,0.0159,16733.10,0.005,16588.19,0.0024,16733.12,0.005
3,1671462570761,BTC_USD,16701.10,0.0250,16712.49,0.0250,16701.09,0.0002,16712.50,0.2883,...,16733.10,0.005,16582.92,0.0326,16733.12,0.005,16580.22,0.0030,16733.19,0.005
4,1671462571780,BTC_USD,16701.10,0.0250,16712.48,0.0003,16701.09,0.0002,16712.49,0.0250,...,16733.07,0.005,16588.19,0.0024,16733.10,0.005,16582.92,0.0326,16733.12,0.005


<h2>Placing market orders</h2>

In [22]:
#Assumption:
#Orders beyond the liquidity of the orderbook
#will be executed at the worst price
def place_market_order(side : str = "buy", dollar_value : float = 10000):
    
    price = []

    for row in df2.to_dict(orient="records"):
        
        side = "ask" if side == "buy" else "bid"
            
        total_price = 0
        total_qty = 0

        for i in range(20):
            if(total_price < dollar_value):
                total_price += row[f"{side}_{i}_px"] * row[f"{side}_{i}_qty"]
                total_qty += row[f"{side}_{i}_qty"]
                last = str(i)
                
#         print(f"Total {order_type}: {total_price}")
        
        if total_price >= dollar_value:
            extra = total_price - dollar_value
            total_qty = total_qty - extra / row[f"{side}_{last}_px"]
        else:
            extra = dollar_value - total_price
            total_qty = total_qty + extra / row[f"{side}_{last}_px"]

#         print(total_price)
        
        avg_price = dollar_value / total_qty
        price.append(avg_price)
    print(price)

In [23]:
place_market_order("sell", 2000)

[16695.203138698194, 16695.20208690026, 16695.213255999322, 16696.44119674212, 16696.44119674212]


In [24]:
# from functools import partial

# best_bid = -1
# limit_orders = []

# # define your callback function on message received
# def check(conn: Connection, response: WebSocketResponse, x):
#     global best_bid
#     global orders
    
#     print(x)
    
#     if response.error:
#         print('error returned: ', response.error)
#         conn.close()
#         return
    
#     data = response.result
    
#     if list(data.keys())[0] != "status":
#         best_bid_local = data['bids'][0][0]
#         print(best_bid)
#         best_bid = best_bid_localbb
        
        
# async def order_stream():
#     channel_name = "BTC_USD"
#     depth = "5"
#     update_rate = "1000ms"
    
#     add_part = partial(check, x = 5)
    
#     conn = Connection(Configuration())

#     channel = SpotOrderBookChannel(conn, add_part)
    
#     channel.subscribe([channel_name, depth, update_rate])
    
#     await conn.run()
    
     

# start_time = time.perf_counter()
# await order_stream()
# end_time = time.perf_counter()
# print(f"Time: {end_time - start_time}")

In [63]:
best_bid = -1
limit_orders = []

def check(conn: Connection, response: WebSocketResponse):
    global best_bid
    global limit_orders
    
    side = limit_orders[0][0]
    price = float(limit_orders[0][1])
    qty = float(limit_orders[0][2])
    
    if response.error:
        print('error returned: ', response.error)
        conn.close()
        return
    
    data = response.result
    
    if list(data.keys())[0] == "status":
        return 
    
    best_local = float(data[f"{side}"])
    print(f"order_price: {price}, best_{side}_local = {best_local}")
    
    if side == "b" and price < best_local:
        print(f"Order executed at price: {best_local}")
        conn.close()
        return
    
    if side == "a" and price > best_local:
        print(f"Order executed at price: {best_local}")
        conn.close()
        return

        
        
async def order_stream():
    channel_name = "BTC_USD"
    depth = "5"
    update_rate = "1000ms"
    
    conn = Connection(Configuration())

    channel = SpotBookTickerChannel(conn, check)
    
    channel.subscribe([channel_name])
    
    await conn.run()

In [64]:
#Assumption
#Once best bid / ask price crosses the limit price, we assume the entire quantity is executed
async def place_limit_order(side : str = "sell", price : float = 20000, qty : float = 1):
    side = "a" if side == "buy" else "b"
    limit_orders.append([side, price, qty])
    await order_stream()

In [65]:
await place_limit_order("bid", 17220, 1)

order_price: 17220.0, best_b_local = 17249.62
Order executed at price: 17249.62


In [74]:
market_orders = []
best_global = -1
NUM_OF_SNAPSHOTS = 1
curr_snapshot = 0
orders = []
market_avg_price = 0
def market_check(conn: Connection, response: WebSocketResponse):
    # global curr_snapshot 
    # global NUM_OF_SNAPSHOTS
    # global orders
    # if response.error:
    #     print('error returned: ', response.error)
    #     conn.close()
    #     return
    
    # data = response.result
    # if list(data.keys())[0] != "status":
    #     orders.append(data)
    #     curr_snapshot += 1
    
    # if curr_snapshot >= NUM_OF_SNAPSHOTS:
    #     conn.close()
    global curr_snapshot
    global NUM_OF_SNAPSHOTS
    global market_avg_price
    if response.error:
        print('error returned: ', response.error)
        conn.close()
        return
    
    data = response.result
    print(data)
    if list(data.keys())[0] != "status":
        curr_snapshot += 1
    
    if curr_snapshot >= NUM_OF_SNAPSHOTS:
        global market_orders
        global best_global
    #     print(limit_orders)
        side = market_orders[0][0]
        dollar_value = float(market_orders[0][1])
        price = []

            
        side = "asks" if side == "buy" else "bids"
        
        total_price = 0
        total_qty = 0


        for i in range(20):
            if(total_price < dollar_value):
                total_price += float(data[side][i][0]) * float(data[side][i][1])
                total_qty += float(data[side][i][1])
                last = i
                    
    #         bprint(f"Total {order_type}: {total_price}")
            
        if total_price >= dollar_value:
            extra = total_price - dollar_value
            total_qty = total_qty - extra / float(data[side][last][0])
        else:
            extra = dollar_value - total_price
            total_qty = total_qty + extra / float(data[side][last][0])

    #         print(total_price)
            
        avg_price = dollar_value / total_qty
        market_avg_price = avg_price
        print(avg_price)
        conn.close()
        return 
        
    
#     print(data)
#     time.sleep(1000)
    
        
        
async def market_order_stream():
    channel_name = "BTC_USD"
    depth = "20"
    update_rate = "1000ms"
    
    conn = Connection(Configuration())

    channel = SpotOrderBookChannel(conn, market_check)
    # channel = SpotBookTickerChannel(conn, check)
    
    channel.subscribe([channel_name, depth, update_rate])
    # channel.subscribe([channel_name])
    
    await conn.run()

In [75]:
async def place_market_order(side = "sell", price = 20000):
    # side = "a" if side == "buy" else "b"
    market_orders.append([side, price])
    await market_order_stream()

In [76]:
await place_market_order("sell", 2000)

{'status': 'success'}
{'t': 1673297377131, 'lastUpdateId': 913943532, 's': 'BTC_USD', 'bids': [['17191.1', '0.2959'], ['17189.23', '0.0016'], ['17185.5', '0.5567'], ['17177.4', '0.0032'], ['17162.85', '0.0014'], ['17162.84', '0.0022'], ['17162.82', '0.0004'], ['17162.81', '0.5454'], ['17162.8', '0.0002'], ['17155.28', '0.014'], ['17139.49', '0.0024'], ['17128.3', '1.8791'], ['17100', '0.0016'], ['17088.99', '0.003'], ['17088.98', '0.0146'], ['17087.4', '0.0002'], ['17082', '1.1163'], ['17069.59', '0.0024'], ['17051.28', '0.0034'], ['17044.74', '0.7308']], 'asks': [['17209.43', '0.003'], ['17209.45', '0.003'], ['17209.47', '0.003'], ['17209.49', '0.003'], ['17209.51', '0.003'], ['17209.53', '0.003'], ['17209.55', '0.003'], ['17209.57', '0.003'], ['17209.59', '0.003'], ['17209.61', '0.003'], ['17209.63', '0.003'], ['17209.65', '0.003'], ['17209.67', '0.003'], ['17209.69', '0.003'], ['17209.7', '0.2959'], ['17214.47', '0.0004'], ['17214.48', '0.003'], ['17214.5', '0.003'], ['17214.52', '0

In [77]:
async def twap(start_time, end_time, steps, side = "sell", quantity = 1000):
    global curr_snapshot
    curr_snapshot = 0
    interval = (end_time-start_time)/steps
    executed_prices = []
    curr_time = start_time
    interval_quantity = quantity/steps
    await place_market_order(side, interval_quantity)
    executed_prices.append(market_avg_price)
    curr_time+=interval
    while(curr_time<end_time):
        curr_snapshot = 0
        time.sleep(interval)
        await place_market_order(side, interval_quantity)
        executed_prices.append(market_avg_price)
        curr_time+=interval
    curr_snapshot = 0
    return sum(executed_prices)/len(executed_prices)

In [78]:
await twap(1671464378, 1671464378+100, 5, side = "sell", quantity = 1000)

{'status': 'success'}
{'t': 1673297428013, 'lastUpdateId': 913946325, 's': 'BTC_USD', 'bids': [['17189.5', '0.2959'], ['17189.23', '0.0016'], ['17185.5', '0.5567'], ['17179.75', '0.0278'], ['17162.82', '0.0018'], ['17162.81', '0.5454'], ['17162.8', '0.0002'], ['17152.93', '0.003'], ['17147.09', '0.0075'], ['17139.49', '0.0024'], ['17128.3', '1.8791'], ['17100', '0.0016'], ['17088.98', '0.0146'], ['17087.4', '0.0002'], ['17082.01', '0.003'], ['17082', '1.1163'], ['17069.59', '0.0024'], ['17051.28', '0.0034'], ['17044.74', '0.7308'], ['17012', '0.0002']], 'asks': [['17205.67', '0.003'], ['17205.69', '0.003'], ['17205.71', '0.003'], ['17205.73', '0.003'], ['17205.74', '0.0058'], ['17205.75', '0.003'], ['17205.77', '0.003'], ['17205.79', '0.003'], ['17205.81', '0.003'], ['17205.83', '0.003'], ['17205.85', '0.003'], ['17205.87', '0.003'], ['17205.89', '0.003'], ['17208.49', '0.003'], ['17208.5', '0.2959'], ['17209.19', '0.003'], ['17209.21', '0.003'], ['17209.23', '0.003'], ['17209.25', '0.

17188.558936072546